In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 7.9 MB/s 
     |████████████████████████████████| 596 kB 78.4 MB/s 
     |████████████████████████████████| 880 kB 33.6 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 6.6 MB 54.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=61c6e42c6ca26f54003e548e676e93791a5b3f3b3a0461fb17a7d3b23321fbcd
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# !pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/Shareddrives/DL

Mounted at /content/gdrive/
/content/gdrive/Shareddrives/DL


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertTokenizer
from transformers import PLBartTokenizer, PLBartForConditionalGeneration
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from transformers import T5Tokenizer, T5Model,T5ForConditionalGeneration

# plb_model =  PLBartForConditionalGeneration.from_pretrained("PLBART Model",max_position_embeddings=1500,ignore_mismatched_sizes=True).to(device)
t5_model = T5ForConditionalGeneration.from_pretrained("CodeT5FinalModel",dropout_rate=0.5,d_model=1850,ignore_mismatched_sizes=True).to(device)

t5_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 1850)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 1850)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1850, out_features=512, bias=False)
              (k): Linear(in_features=1850, out_features=512, bias=False)
              (v): Linear(in_features=1850, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=1850, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.5, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1850, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=1850, bias=False)
              (dropo

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')

def RobertaTokenizerForSeq2SeqTest(x):
  return tokenizer.batch_encode_plus(x,max_length=1000,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

Downloading:   0%|          | 0.00/687k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

In [8]:
tc2code = pd.read_json('./ResultValidation/data_testing.jsonl',lines=True)

In [9]:
tc2code.head()

,testCode,sourceCode
0,package com.example.util;\n\nimport com.exampl...,package com.example.util;\n\nimport com.exampl...
1,package com.example;\n\nimport org.junit.Test;...,package com.example;\n\nimport com.example.mod...
2,package com.example.web;\n\nimport com.example...,package com.example.web;\n\nimport com.example...
3,package com.example.service;\n\nimport com.exa...,package com.example.service;\n\nimport com.exa...
4,package com.practicalunittesting.chp03.money;\...,package com.practicalunittesting.chp03.money;\...


In [10]:
X_test, y_test = tc2code['testCode'], tc2code['sourceCode']

In [11]:
print(len(X_test.to_list()))

10


In [12]:
print(type(RobertaTokenizerForSeq2SeqTest(X_test.to_list()[1:2])))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [13]:
print(y_test.to_list()[0])

package com.example.util;

import com.example.model.ToDo;

public class PayloadValidator {

    public static boolean validateCreatePayload(ToDo toDo) {
        return toDo.getId() <= 0;
    }

}



In [14]:
from torch.utils import data
class CodeTCDatasetTest(data.Dataset):
    def __init__(self, testTensor,expectedSourceCode,testCodeInput):
      self.testTensor=testTensor
      self.expectedSourceCode=expectedSourceCode
      self.testCodeInput=testCodeInput


    def __getitem__(self, index):
        return self.testTensor[index],self.expectedSourceCode[index],self.testCodeInput[index]


    def __len__(self):
        return len(self.testCodeInput)


def collate_fn(data):
    testTensorList,expectedSourceCode,testCodeInput = zip(*data)

    testTensors = torch.stack(testTensorList, 0)
      
    return testTensors, expectedSourceCode, testCodeInput

def get_test_loader(testTensor,expectedSourceCode,testCodeInput,batch_size):
    code2tctest = CodeTCDatasetTest(testTensor,expectedSourceCode,testCodeInput)
    data_loader = torch.utils.data.DataLoader(dataset=code2tctest, 
                                              batch_size=batch_size,
                                              collate_fn=collate_fn)
    return data_loader

In [15]:
X_test_tensors=RobertaTokenizerForSeq2SeqTest(X_test.to_list())
input_ids_test = X_test_tensors['input_ids']

In [16]:
print(len(input_ids_test))

10


In [17]:
batch_size=64
test_loader=get_test_loader(input_ids_test,y_test.to_list(),X_test.to_list(),batch_size)

In [18]:
print(type(X_test_tensors))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [19]:
def getPredictionsGreedy(inputs,model,device):
      greedy_output = model.generate(inputs.to(device), max_length=1800)
      sequences=[]
      for i in range(greedy_output.shape[0]):
        sequences.append(tokenizer.decode(greedy_output[i], skip_special_tokens=True))
      return sequences

def getPredictionsTopK(inputs,model,device):
      sample_outputs = model.generate(
        inputs.to(device),
        do_sample=True, 
        max_length=1800, 
        top_k=900, 
        top_p=0.95, 
        num_return_sequences=3)
    # outputs = model.generate(input_ids)
      sequences=[]
      for i in range(sample_outputs.shape[0]):
        sequences.append(tokenizer.decode(sample_outputs[i], skip_special_tokens=True))
      return sequences

In [20]:
def getPredictedTexts(dataloader_test,genAlgo,model,device,targIters=1,need_targ_iters=False):
    predictions=[]
    allExpectedprogs=[]
    allIpProgs=[]
    idx=0
    with torch.no_grad():
      for ip,estr,istr in dataloader_test:
          if idx>=targIters and need_targ_iters:
            break
          print(f"iter{idx} of {len(dataloader_test)}")
          prediction=genAlgo(ip,model,device)
          predictions=predictions+prediction
          expected=[x for x in estr]
          inputs=[x for x in istr]
          allExpectedprogs=allExpectedprogs+expected
          allIpProgs=allIpProgs+inputs
          idx+=1
    return allExpectedprogs,predictions,allIpProgs

In [21]:
allExpectedprogsGreedy,predictionsGreedy,ipTestCode=getPredictedTexts(test_loader,getPredictionsGreedy,t5_model,device)

iter0 of 1


In [22]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [23]:
print(allExpectedprogsGreedy[0])

package com.example.util;

import com.example.model.ToDo;

public class PayloadValidator {

    public static boolean validateCreatePayload(ToDo toDo) {
        return toDo.getId() <= 0;
    }

}



In [24]:
print(ipTestCode[0])

package com.example.util;

import com.example.model.ToDo;
import org.junit.Test;

import static org.junit.Assert.assertEquals;

public class PayloadValidatorTest {

    @Test
    public void validatePayLoad() {
        ToDo toDo = new ToDo(1, "Sample ToDo 1", true);
        assertEquals(false, PayloadValidator.validateCreatePayload(toDo));
    }

    @Test
    public void validateInvalidPayLoad() {
        ToDo toDo = new ToDo(0, "Sample ToDo 1", true);
        assertEquals(true, PayloadValidator.validateCreatePayload(toDo));
    }


}



In [25]:
print(predictionsGreedy[0])

/*
 * Copyright (C) 2001-2009 VTT Technical Research Centre of Finland.
 *
 *
 * This library is free software; you can redistribute it and/or
 * modify it under the terms of the GNU Lesser General Public
 * License as published by the Free Software Foundation;
 * version 2.1 of the License.
 *
 * This library is distributed in the hope that it will be useful,
 *
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
 * Lesser General Public License for more details.
 *
 *
 * * You should should should should should should should should should should should should should should should should should should should should should should should should should should
 *
 *
 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

In [26]:
print(predictionsGreedy[1])

/*
 * IFX-Framework - IFX XML to JavaBean application framework.
 * Copyright (C) 2003  The IFX-Framework Team
 * 
 * This library is free software; you can redistribute it and/or
 * modify it under the terms of the GNU Lesser General Public
 * License as published by the Free Software Foundation; either
 * version 2.1 of the License, or (at your option) any later version.
 * 
 * This library is distributed in the hope that it will be useful,
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
 * Lesser General Public License for more details.
 * 
 * You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You You 

Predictions are super bad, so how does t5 fare with top 5

In [27]:
predictiondf_t5 = pd.DataFrame(list(zip(allExpectedprogsGreedy, predictionsGreedy,ipTestCode)), columns =['Expected', 'predictions','ipUnitTest']) 

In [28]:
predictiondf_t5.to_json('ResultValidation/predictions_tesing_codet5.jsonl',orient='records', lines=True)

In [29]:
allExpectedprogsGreedy,predictionsGreedy,ipTestCode=getPredictedTexts(test_loader,getPredictionsTopK,t5_model,device,1,True)
predictiondf_t5 = pd.DataFrame(list(zip(allExpectedprogsGreedy, predictionsGreedy,ipTestCode)), columns =['Expected', 'predictions','ipUnitTest']) 
predictiondf_t5.to_json('ResultValidation/predictions_codet5_testing_set2.jsonl',orient='records', lines=True)

iter0 of 1


In [30]:
predictionsGreedy[0]

'/**\n * * Copyright (c) 2000-2012 Liferay, Inc. All rights reserved.\n *\n * This library is free software; you can redistribute it and/or modify it under\n * the terms of the GNU Lesser General Public License as published by the Free\n * Software Foundation; either version 2.1 of the License, or (at your option)\n * any later version.\n *\n * This library is distributed in the hope that it will be useful, but WITHOUT\n * ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS\n * FOR A PARTICULAR PURPOSE. See the GNU Lesser General Public License for for\n *\n *\n * details.\n */\n\n\npackage com.liferay.portal.security;\n\nimport com.liferay.portal.kernel.....7..orm..._\n\n\n/**\n * @ @ @ @ @ @ @ @ @ @ @ @ @ @ @\n */\npublicpublic class UpgradeProcess extends UpgradeProcess {\n\n\n\t\t___________________________________________________________________________________________________________________________________________________________________________________

Still bad :(